In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st
from scipy.stats import chi2_contingency, chi2 
import seaborn as sns
import matplotlib.pyplot as plt
import pingouin as pg

In [2]:
logs = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-rom-fadeev/KC_case_data .csv')
logs.head(5)

/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedCacheFailedWarning: Failed to use cache while checking for outdated package.
Set the environment variable OUTDATED_RAISE_EXCEPTION=1 for a full traceback.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs
/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs
/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.4.0, the latest is 0.5.5.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [3]:
# logs = logs.query("device_id ==4014 and event == 'app_start'" )
#             .groupby(["utm_source"], as_index=False)\
#             .agg({'event' : 'count'})\
#             .sort_values('event', ascending=False)
# logs

In [40]:
logs['date'] = pd.to_datetime(logs['date'])

# inst['device_id'].nunique() == 154597 и count() такой же
inst = logs.query("event == 'app_install'" )\
    .groupby(["device_id","date"], as_index=False)\
    .agg({'event' : 'count'})
inst = inst.rename(columns={'event': "install",'date': "date_install" })

# purch['device_id'].nunique() == 70875   purch['device_id'].count() == 141383
# тк покупок у одного id может быть несколько в разные даты , убираем дубликаты
purch = logs.query("event == 'purchase'" )\
    .groupby(["device_id","date"], as_index=False)\
    .agg({'event' : 'count'}) 
purch = purch.rename(columns={'event': "purchase",'date': "date_purchase"})
# тк главное- это первая покупка, то удаляем дубликаты покупок по id
purch = purch.drop_duplicates(subset = ['device_id'])

all_ev = inst.merge(purch, how='left', on='device_id')
#  7 дней с даты установки
all_ev['day'] = (all_ev['date_purchase'] - all_ev['date_install']).dt.days
all_ev['day'] = all_ev['day'].fillna(0)
all_ev['delta']=all_ev['date_install'] + pd.Timedelta(days=7)

# только покупки             
purch1 = all_ev.query("day<=7.0 and purchase != 'NaN' ")\
    .groupby(['date_install'], as_index=False)\
    .agg({'device_id' : 'nunique'})
# все установки без  day<=7.0 тк неважно купили или нет, нужно общее кол-во установок
inst1 = all_ev.groupby(['date_install'], as_index=False)\
    .agg({'install' : 'count'})

com = purch1.merge(inst1, how='left', on='date_install') 
com = com.rename(columns={'device_id': "purchase"})
com['CR'] = ((com['purchase']/com['install'])*100).round(1) 
com = com.sort_values('CR', ascending=False)
com

,date_install,purchase,install,CR
0,2020-01-01,1408,3579,39.3
8,2020-01-09,558,1424,39.2
14,2020-01-15,1650,4310,38.3
13,2020-01-14,1973,5173,38.1
1,2020-01-02,1186,3144,37.7
...,...,...,...,...
77,2020-03-18,235,1171,20.1
88,2020-03-29,223,1117,20.0
81,2020-03-22,251,1261,19.9
87,2020-03-28,209,1091,19.2


In [5]:
# inst['device_id'].count()

In [6]:
# inst['device_id'].nunique()

In [7]:
# inst.drop_duplicates(subset = ['device_id']).count()
# inst = logs.query("event == 'app_install'" ).drop_duplicates(subset = ['device_id'])
# inst['device_id'].count()

In [8]:
# logs.drop_duplicates(subset = ['device_id']).count()

In [9]:
# all_ev['cogorts']=all_ev['date_install'] + pd.Timedelta(days=7)


In [10]:
# all_ev['ggggg'] = all_ev['cogort'] <= all_ev['date_install'] <= all_ev['delta']

In [11]:
# logs['date'] = pd.to_datetime(logs['date'])

# # inst['device_id'].nunique() == 154597 и count() такой же
# inst = logs.query("event == 'app_install'" )\
#     .groupby(["device_id","date"], as_index=False)\
#     .agg({'event' : 'count'})
# inst = inst.rename(columns={'event': "install",'date': "date_install" })

# # purch['device_id'].nunique() == 70875   purch['device_id'].count() == 141383
# purch = logs.query("event == 'purchase'" )\
#     .groupby(["device_id","date"], as_index=False)\
#     .agg({'event' : 'count'}) 
# purch = purch.rename(columns={'event': "purchase",'date': "date_purchase"})

# all_ev = inst.merge(purch, how='left', on='device_id') 
# all_ev['day'] = (all_ev['date_purchase'] - all_ev['date_install']).dt.days
# all_ev['day'] = all_ev['day'].fillna(0)
# all_ev['cogort'] = all_ev['date_install'] + pd.Timedelta(days=7)
# all_ev['delta'] = pd.Timedelta(all_ev['date_install'], all_ev['cogort'])

# purch = all_ev.query("day<=7.0 and purchase != 'NaN'")\
#     .groupby(['date_install'], as_index=False)\
#     .agg({'device_id' : 'nunique'})

# inst = all_ev.groupby(['date_install'], as_index=False)\
#     .agg({'install' : 'count'})

# com = purch.merge(inst, how='inner', on='date_install') 
# # com = com.rename(columns={'device_id': "purchase"})
# # com['CR'] = ((com['purchase']/com['install'])*100).round(1) 
# # com = com.sort_values('CR', ascending=False)
# all_ev